In [1]:
import json
from hashlib import sha256
from base64 import b64decode
import requests
import numpy as np
import sys
import pandas as pd


def rippled_bs58(key):
    '''
    Returns a string that is encoded using the rippled base58 alphabet.
    '''
    alphabet = b'rpshnaf39wBUDNEGHJKLM4PQRST7VWXYZ2bcdeCg65jkm8oFqi1tuvAxyz'
    string = b""
    while key:
        key, idx = divmod(key, 58)
        string = alphabet[idx:idx+1] + string
    return string

def unl_parser(address='https://vl.xrplf.org'):
    '''
    Download the UNL and base64 decode the blob.
    Defaults to https://vl.ripple.com
    Individual validation keys are then constructed from the decoded blob
    payload and the double sha256 hashed checksums. Keys are then base58 encoded.
    '''
    keys = {'status': 'Error',
            'error': False,
            'http_code': '',
            'public_validation_keys': [],
            'expiration': '',}

    try:
        unl = requests.get(address)
        keys['http_code'] = unl.status_code
        unl.raise_for_status()
    except requests.exceptions.RequestException:
        keys['error'] = "Invalid URL: {}.".format(address)
        return pd.DataFrame(keys['error'], columns=['error'])

    try:
        blob = json.loads(b64decode(unl.json()['blob']).decode('utf-8'))
        validators = blob['validators']
        '''
        Convert Ripple Epoch to Unix Epoch
        Reference : https://xrpl.org/basic-data-types.html
        '''
        keys['expiration'] = blob['expiration'] + 946684800

    except json.decoder.JSONDecodeError:
        keys['error'] = "Invalid or malformed manifest."
        return pd.DataFrame(keys['error'], columns=['error'])

    if not validators:
        keys['error'] = "List of validator keys was empty."
        return pd.DataFrame(keys['error'], columns=['error'])

    for i in validators:
        payload = "1C" + i['validation_public_key']
        keys['public_validation_keys'].append(
            rippled_bs58(int(payload + sha256(bytearray.fromhex(sha256(
                bytearray.fromhex(payload)).hexdigest())).hexdigest()[0:8], 16)).decode('utf-8'))
#         print(rippled_bs58(int(payload + sha256(bytearray.fromhex(sha256(
#                 bytearray.fromhex(payload)).hexdigest())).hexdigest()[0:8], 16)).decode('utf-8'))
#     keys['status'] = 'Success'
    
    return pd.DataFrame(keys['public_validation_keys'], columns=['validator_keys'])

In [2]:
vl = unl_parser("https://vl.ripple.com")
vl = vl.sort_values(by=['validator_keys']).reset_index(drop=True)
vl_quantity=vl['validator_keys'].count()

print(vl_quantity)

vl.head(5)

34


,validator_keys
0,nHB8QMKGt9VB4Vg71VszjBVQnDW3v3QudM4DwFaJfy96bj...
1,nHBdXSF6YHAHSZUk7rvox6jwbvvyqBnsWGcewBtq8x1XuH...
2,nHBgiH2aih5JoaL3wbiiqSQfhrC21vJjxXoCoD2fuqcNbr...
3,nHBidG3pZK11zQD6kpNDoAhDxH6WLGui6ZxSbUx7LSqLHs...
4,nHDB2PAPYqF86j9j3c6w1F1ZqwvQfiWcFShZ9Pokg9q4oh...


In [3]:
xrplf = unl_parser("https://vl.xrplf.org")
xrplf = xrplf.sort_values(by=['validator_keys']).reset_index(drop=True)
xrplf_quantity=xrplf['validator_keys'].count()

print(xrplf_quantity)

xrplf.head(5)

34


,validator_keys
0,nHB8QMKGt9VB4Vg71VszjBVQnDW3v3QudM4DwFaJfy96bj...
1,nHBdXSF6YHAHSZUk7rvox6jwbvvyqBnsWGcewBtq8x1XuH...
2,nHBgiH2aih5JoaL3wbiiqSQfhrC21vJjxXoCoD2fuqcNbr...
3,nHBidG3pZK11zQD6kpNDoAhDxH6WLGui6ZxSbUx7LSqLHs...
4,nHDB2PAPYqF86j9j3c6w1F1ZqwvQfiWcFShZ9Pokg9q4oh...


In [4]:
coil = unl_parser("https://vl.coil.com")
coil = coil.sort_values(by=['validator_keys']).reset_index(drop=True)
coil_quantity=vl['validator_keys'].count()

print(coil_quantity)

coil.head(5)

34


,validator_keys
0,nHB8QMKGt9VB4Vg71VszjBVQnDW3v3QudM4DwFaJfy96bj...
1,nHBdXSF6YHAHSZUk7rvox6jwbvvyqBnsWGcewBtq8x1XuH...
2,nHBgiH2aih5JoaL3wbiiqSQfhrC21vJjxXoCoD2fuqcNbr...
3,nHBidG3pZK11zQD6kpNDoAhDxH6WLGui6ZxSbUx7LSqLHs...
4,nHDB2PAPYqF86j9j3c6w1F1ZqwvQfiWcFShZ9Pokg9q4oh...


In [5]:
#Print validators that are present in only one or two of the lists

diff = pd.concat([vl,xrplf,coil]).drop_duplicates(keep=False)

diff.head(50)

,validator_keys


In [6]:
#are the UNLs equal?

print(vl['validator_keys'].equals(xrplf['validator_keys']))
print(xrplf['validator_keys'].equals(coil['validator_keys']))

True
True
